In [ ]:


import pandas as pd

# to generate this, click on goodreads "my books".  Then click "print".  Then set the &per_page=1000 query param
books = pd.read_html("book_tables.txt")[0]




books



In [ ]:
books["title"] = books["title"].str.replace("title ", "")


books.loc[0]["rating"]



In [ ]:


ratings_string = books["rating"].str.extract("(\[.+\])")


books["rating"] = ratings_string[0].str.slice(start=2, stop=3)


books = books[["title", "rating"]]



titles = pd.read_json("books_titles.json")
titles["book_id"] = titles["book_id"].astype(str)



from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

tfidf = vectorizer.fit_transform(titles["mod_title"])



import re
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(query,vectorizer):
    processed = re.sub("[^a-zA-Z0-9 ]", "", query.lower())
    query_vec = vectorizer.transform([query])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -10)[-10:]
    results = titles.iloc[indices]
    results = results.sort_values("ratings", ascending=False)
    
    return results.iloc[0,:]["book_id"]



books["book_id"] = books["title"].apply(lambda x: search(x, vectorizer))
books["user_id"] = "-1"
books = books[["user_id", "book_id", "rating", "title"]]
books = books[~pd.isnull(books["rating"])]


books.to_csv("my_books.csv")

